In [10]:
import torch
from torch import nn
import numpy as np

In [37]:
random_tensor = torch.randint(0, 5, (3,))
acc_tensor = torch.Tensor()

In [39]:
for i in range(4):
    concated_tensor = torch.cat((concated_tensor, random_tensor))

In [40]:
concated_tensor

tensor([0., 0., 3., 0., 0., 3., 0., 0., 3., 0., 0., 3.])

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [11]:
class TransformerRecreation(nn.Module):
    def __init__(self, h, d_k, d_v, d_model, n_encoder_layers, n_decoder_layers, d_ff):
        super().__init__()
        self.h = h
        self.d_k = d_k
        self.d_v = d_v
        self.d_model = d_model
        self.n_encoder_layers = n_encoder_layers
        self.n_decoder_layers = n_decoder_layers
        self.d_ff = d_ff
        
        
        self.W_q_list = [torch.Tensor(1, self.d_model, self.d_k) for i in range(self.h)]
        self.W_k_list = [torch.Tensor(1, self.d_model, self.d_k) for i in range(self.h)]
        self.W_v_list = [torch.Tensor(1, self.d_model, self.d_v) for i in range(self.h)]
        
        self.W_out = torch.Tensor(1, self.h * self.d_v, self.d_model)
        
        self.ff_model = nn.Sequential(
            nn.Linear(self.d_model, self.d_ff),
            nn.ReLU(),
            nn.Linear(self.d_ff, self.d_model)
        )
        
    def encoder_layer(self, x):
        x_original = x
        x = multiHeadAttention(x)
        x = nn.LayerNorm(x + x_original)
        sublayer1_output_original = x
        x = positionFeedForward(x)
        x = nn.LayerNorm(x + sublayer1_output_original)
        return x
    
    def stack_encoder_layers(self, x):
        for i in range(self.n_encoder_layers):
            x = encoder_layer(x)
        return x

    def decoder_layer(self, x, encoder_output):
        x_original = x
        ### Sublayer 1: Masked Multi-head Attention
        x = multiHeadAttention(x)
        x = nn.LayerNorm(x + x_original)
        sublayer1_output_original = x
        
        ### Sublayer 2: multi-head attention
        x = multiHeadAttention(encoder_output) # multiHeadAttention(x)  ### are encoder output and sublayer1_output_original both put in this?
        x = nn.LayerNorm(x + sublayer1_output_original)
        sublayer2_output_original = x
        
        ### Sublayer 3: Feed Forward
        x = positionFeedForward(x)   
        x = nn.LayerNorm(x + sublayer2_output_original)
        return x
        

    def stack_decoder_layers(self, x, encoder_output):
        for i in range(self.n_decoder_layers):
            x = decoder_layer(x, encoder_output)
        return x

    
    def attention(self, Q, K, V):
        x = torch.matmul(Q, K)
        x = x / np.sqrt(self.d_k)
        x = nn.Softmax(x)
        x = nn.matmul(x, V)
        return x # n x d_v

    def multiHeadAttention(self, Q):
        K = torch.Tensor(1, self.d_k, self.d_k)
        V = torch.Tensor(1, self.d_v, self.d_v)

        acc_tensor = torch.Tensor()
        for i in range(self.h):
            # project q,k,v matrices from d_model to d_k, d_v
            proj_q = torch.matmul(Q, self.W_q_list[i])
            proj_k = torch.matmul(K, self.W_k_list[i])
            proj_v = torch.matmul(V, self.W_v_list[i])
            
            # calculate attention
            head = attention(proj_q, proj_k, proj_v)
            
            # concat head_i
            acc_tensor = torch.cat(acc_tensor, head)
        
        return torch.matmul(acc_tensor, self.W_out)
    
    def positionFeedForward(self, x):
        x = self.ff_model(x)
        return x
        
    def positional_encoding(self, pos, i):
        return np.sin(pos / 10_000**(2*i / self.d_model))
    
    def forward(self, x):
        x = encoder_layer(x)
        x = decoder_layer(x)
        encoder_output = x
        

In [16]:
# model = TransformerRecreation().to(device)
# print(model)
# inputs = torch.rand(1, 28, 28, device = device)
# pred = model(inputs)
# print(pred)

# pred_prob = nn.Softmax(dim=1)(pred)
# pred_prob

# y_pred = pred_prob.argmax(1)
# y_pred

tensor([[-0.0256, -0.0691,  0.0705, -0.0238, -0.0211,  0.0156, -0.0638,  0.0071,
          0.0155,  0.0243]], device='mps:0', grad_fn=<LinearBackward0>)


In [ ]:
2